In [0]:

from pathlib import Path
import torch
from torch.utils.data import TensorDataset ,DataLoader
from torch import nn,optim
import torch.nn.functional as F
import numpy as np # linear algebra
import pandas as pd # data processing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt    
from torchvision import datasets,transforms

In [28]:
from google.colab import files
uploaded1 = files.upload()

Saving train.csv to train (1).csv


In [0]:
train = pd.read_csv("train.csv")

In [0]:
test = pd.read_csv("test.csv")


In [68]:
x=train.drop("label",axis=1)
y=np.array(train['label'])
x.shape,y.shape

((42000, 784), (42000,))

In [67]:
torch_X_train = torch.from_numpy(x.values).type(torch.FloatTensor)/255               #changing the train test values into tesor and normalised them by divideing them by255
torch_y_train = torch.from_numpy(y).type(torch.LongTensor)
myDataset = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
valid_no  = int(0.2 * len(myDataset))
# so divide the data into trainset and testset
trainSet,testSet = torch.utils.data.random_split(myDataset,(len(myDataset)-valid_no,valid_no))
print(f"len of trainSet {len(trainSet)} , len of testSet {len(testSet)}")
batch_size=64                                                                   #here we are dividing the train data in 64 batches
train_loader  = DataLoader(trainSet , batch_size=batch_size ,shuffle=True)      #uploading the batches into the loadeers
test_loader  = DataLoader(testSet , batch_size=batch_size ,shuffle=True)

len of trainSet 33600 , len of testSet 8400


In [0]:
from torch import nn, optim
class Network(nn.Module):
    def __init__(self):
        super().__init__()                                                       #were are creating neural networks
        self.fc1 = nn.Linear(784, 256)    #input
        self.fc2 = nn.Linear(256, 128)    #hidden
        self.fc3 = nn.Linear(128, 64)     #hidden
        self.fc4 = nn.Linear(64, 10)      #output

        # Dropout module with 0.2 drop probability
        self.dropout = nn.Dropout(p=0.2)             

    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)

        # Now with dropout
        x = self.dropout(F.relu(self.fc1(x)))           #giving activation fuc as relu to input and hidden layer
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))

        # output so no dropout here
        x = F.log_softmax(self.fc4(x), dim=1)           #to move from hidden to output layer you are using logsoftMax

        return x
        
model=Network()
optimizer=torch.optim.SGD(model.parameters(), lr=0.05, weight_decay= 1e-4, momentum = 0.8,nesterov = True)
criterion=nn.CrossEntropyLoss()

In [65]:

epochs=10                                                         #here is training and testing 
train_losses,test_losses=[],[]                                   
for e in range(epochs):
    running_loss=0
    for images,labels in train_loader:
        optimizer.zero_grad()    #initializing with 0 all the gradients
        log_ps=model(images)     #putting the batches into model  
        loss=criterion(log_ps,labels)
        loss.backward()               
        optimizer.step()
        running_loss+=loss.item()     #after getting losses in criterion you are summing it up 
        
    else:
        test_loss=0
        accuracy=0
        
        with torch.no_grad():                                 #doing the same process for test data
            model.eval()
            for images,labels in test_loader:
                log_ps=model(images)
                test_loss+=criterion(log_ps,labels)
                ps=torch.exp(log_ps)
                top_p,top_class=ps.topk(1,dim=1)
                equals=top_class==labels.view(*top_class.shape)
                accuracy+=torch.mean(equals.type(torch.FloatTensor))
        model.train()
        train_losses.append(running_loss/len(train_loader))
        test_losses.append(test_loss/len(test_loader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),                             
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))

Epoch: 1/10..  Training Loss: 0.022..  Test Loss: 0.094..  Test Accuracy: 0.976
Epoch: 2/10..  Training Loss: 0.056..  Test Loss: 0.092..  Test Accuracy: 0.974
Epoch: 3/10..  Training Loss: 0.054..  Test Loss: 0.092..  Test Accuracy: 0.975
Epoch: 4/10..  Training Loss: 0.048..  Test Loss: 0.092..  Test Accuracy: 0.973
Epoch: 5/10..  Training Loss: 0.045..  Test Loss: 0.089..  Test Accuracy: 0.976
Epoch: 6/10..  Training Loss: 0.042..  Test Loss: 0.094..  Test Accuracy: 0.973
Epoch: 7/10..  Training Loss: 0.040..  Test Loss: 0.087..  Test Accuracy: 0.975
Epoch: 8/10..  Training Loss: 0.036..  Test Loss: 0.089..  Test Accuracy: 0.975
Epoch: 9/10..  Training Loss: 0.036..  Test Loss: 0.096..  Test Accuracy: 0.974
Epoch: 10/10..  Training Loss: 0.035..  Test Loss: 0.098..  Test Accuracy: 0.975


In [64]:
print("Our model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

Our model: 

 Network(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) 

The state dict keys: 

 odict_keys(['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias', 'fc4.weight', 'fc4.bias'])


In [63]:
test_images = test                                            
test_image = test_images.loc[:,test_images.columns != "label"].values
test_dataset = torch.from_numpy(test_image).type(torch.FloatTensor)/255
print(test_dataset.shape)
#test_dataset = torch.utils.data.TensorDataset(test_dataset)
new_test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle = False)

torch.Size([28000, 784])


In [0]:
results = []                                    #testing the created network with test data on batch size 100 
with torch.no_grad():
    model.eval()
    for images in new_test_loader:
        output = model(images)
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim = 1)
        results += top_class.numpy().tolist()

In [61]:
predictions = np.array(results).flatten()      #showing the predictions of test data
print(predictions[:5])
print(predictions.shape)

[2 0 9 9 3]
(28000,)
